In [137]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [138]:
# this command allows more than one result showed
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [139]:
# dataset
flights = pd.read_csv("flights_DQ.csv")


In [268]:
# 0. examination of the dataset
# flights.shape # dimension of the data
# flights.columns
flights.head()
# flights.dtypes
flights.describe() # missing value; outliers
flights.info() # missing value; column property
flights.describe(include='object')
flights.destination.value_counts(normalize=True) # distribution of each category

date           float64
delay          float64
distance       float64
origin          object
destination     object
dtype: object

In [ ]:
# 1. data preparation [Talk with the data collector in case something goes wrong with the whole pipeline]
# 1.1 Missing data [many manipulations are not available with missing data]

In [321]:
# Q: Is there any missing data for each column?
flights.info()
## flights.isna().any()
# Q: How many missing rows do we have
## flights.isna().apply(sum)
# Q: Get positions of missing data and show samples with missing data
loc = np.where(flights.isna())
flights.iloc[loc[0],]
# Q: Drop all rows with missing data
flights.dropna()
# Q: For delay, fill in missing sites with its average
flights['delay'].fillna(flights['delay'].mean(),inplace = True)
# Q: For origin, fill in missing sites using the second most common origin-desination pair
most_freq_pair = flights.groupby('destination')['origin'].agg(lambda x:x.value_counts().index[0]).to_dict()
missing_dest = flights.loc[flights.origin.isna(),'destination'].tolist()
flights.loc[flights.origin.isna(),'org'] = [most_freq_pair[mis_val] for mis_val in missing_dest]
# Q: Simply drop the rows with missing data
flights.dropna(inplace=True)
#  For date, fill in 

In [284]:
# 1.2 duplicates
# create a duplicated datasets
dup_flights = pd.concat([flights[:4],flights[:4]])
# Q: Check any duplicates
dup_flights.duplicated().any()
# Q: drop duplicated rows
dup_flights.drop_duplicates(inplace=True)

In [80]:
# 1.3 outliers
# Q: calculate Inter Quantile Range(IQR) of distance
d_lower = flights.distance.quantile(0.25)
d_upper = flights.distance.quantile(0.75)
# Q: select subset with distance falls within IQR and the origin as BUR,MSY,PDX,HOU or origin starts with A
flights.query("distance>@d_lower & distance<@d_upper & (origin.isin(['BUR','MSY','PDX','HOU'])|origin.str.startswith('A'))")
# Q: drop rows falls outside of the IQR
flights.drop(flights.index[np.array(flights.distance < d_lower) & np.array(flights.distance > d_upper)])
# or keep rows not dropped
flgihts = flights[(flights.distance > d_lower)&(flights.distance < d_upper)]
# what np.where returns is the position; drop only accepts indexes or column names

,date,delay,distance,origin,destination,converted
1,1022100.0,NaN,239.0,HOU,DAL,1
2,3210808.0,6.0,288.0,BWI,ALB,0
3,NaN,-15.0,NaN,MCI,OKC,1
4,1031304.0,23.0,935.0,PHX,TUL,0
5,1022052.0,26.0,1363.0,NaN,LAX,1
...,...,...,...,...,...,...
992,3261640.0,-6.0,1235.0,LAS,HOU,1
993,3082125.0,113.0,180.0,OAK,RNO,1
997,3170830.0,20.0,1448.0,BNA,PHX,1
998,1041339.0,20.0,1107.0,PHX,SEA,0


In [ ]:
# 1.4 meaningful
# 1.5 balanced for categorical features and targets
# merge some categories into the other
# delete some categories

In [117]:
# 1.5 Data transformation
# Q: for origin, replace BUR with BUT and BWI with nan
flights.origin.replace({'BUR':'BUT','BWI':np.nan}) # pandas replace
# Q: for origin, replace B with A for all city names
flights.origin.apply(lambda x:x.replace('B','A'))  # build-in replace
# Q: bins distance column with its quantiles and store in a new variable
flights['distance_bins'] = pd.qcut(flights.distance,q=4,\
                                   labels=np.quantile(flights.distance,q=np.arange(0,1,0.25)))
# Q: continued with the last question, transform distance_bins into one-hot labels and add as new columns
dumm_vars=['distance_bins']
for var in dumm_vars:
    # change False to True if you use model with a collinearity problem
    dumm_list = pd.get_dummies(flights[var], prefix=var,drop_first=False)
    flights=flights.join(dumm_list)
    flights.drop(var,axis=1,inplace=True)
# Q: 

AttributeError: Can only use .str accessor with string values!

In [135]:
dumm_vars=['distance_bins']
for var in dumm_vars:
    # change False to True if you use model with a collinearity problem
    dumm_list = pd.get_dummies(flights[var], prefix=var,drop_first=False)
    flights=flights.join(dumm_list)
    flights.drop(var,axis=1,inplace=True)

In [136]:
flights.head()

,date,delay,distance,origin,destination,converted,distance_bins_108.0,distance_bins_288.0,distance_bins_371.0,distance_bins_590.0
0,2221605.0,3.0,358.0,BUR,SMF,1,0,1,0,0
2,3210808.0,6.0,288.0,BWI,ALB,0,1,0,0,0
4,1031304.0,23.0,935.0,PHX,TUL,0,0,0,0,1
6,1181550.0,2.0,223.0,BUR,LAS,1,1,0,0,0
7,3041110.0,-3.0,321.0,MSY,BHM,1,0,1,0,0
